# Paso el min_cost en todas las matrices (modos) de neq y guardo:
## - matrices finales
## - eigenvalues
## - ponderaciones
# en "min_cost_assigned" p/ después ser leídos en los scripts de analysis

In [2]:
using DataFrames
using Gadfly
using Cairo
using Distributions
set_default_plot_size(40cm, 14cm)

In [3]:
function read_ptraj_modes(file, modes_elements, nmodes::Int64=0, norma::Bool=true)    

    modes_file=open(file, "r")
    modes_text = readdlm(modes_file, skipstart=0, skipblanks=true, 
    comments=true, comment_char='\*')
    close(modes_file)

    if nmodes == 0
        nmodes = modes_text[1, 5]
    end
    ncoords = convert(Int64, modes_elements)
    lines = ceil(Int64, ncoords/7)
    rest = convert(Int64, ncoords % 7)
    
    eval=Array{Float64}(nmodes);
    mode = Array{Float64}(ncoords, nmodes);
    temp1=Array{Float64}(ncoords, 1);
    temp2 = Array{Float64}(ncoords+(7-rest));

    j=lines + 1 + 2 # 1 p/ q lea la prox linea 2 por el header

    for i=1:nmodes
        eval[i] = modes_text[j, 2]
        temp = permutedims(modes_text[(j+1):(lines+j), :], [2, 1])
        temp2 = reshape(temp, ncoords+(7-rest))
        for k=(rest+1):7
            pop!(temp2)
        end
    mode[:, i] = temp2
        j = j + lines + 1
    end
    
    if norma == true
        for i=1:nmodes
            mode[: ,i] = mode[:, i] / norm(mode[:, i])
        end
    end
    
    return mode, eval
end

read_ptraj_modes (generic function with 3 methods)

In [4]:
# Preparo variables
aa = 101
aa3 = aa * 3
aa3_6 = aa3 - 6
n_frames = 400
main_dir_long = "/home/german/labo/16/pdz_gramm/long/"
neq_dir = "lb_dats/"
neq_modes_dir = "vecs/"
min_cost_modes_dir = "min_cost_assigned/"
min_cost_modes_full_dir = "min_cost_assigned_full/"


neq_pond_template = "-lbnoteq_ca_mode_freq_nohist.dat"       # ej: "1" * neq_pond_template
neq_modes_template = "lb_prod_vecs_ca_"                      # ej: neq_modes_template * "1" * ".dat"

# Leo modos, eigenvalues y ponderaciones correspondientes al 1er frame
pond_file = open(string(main_dir_long, neq_dir, 1, neq_pond_template), "r")
modes_filename = string(main_dir_long, neq_modes_dir, neq_modes_template, 1, ".dat")
neq_modes_1, neq_eigen_1 = read_ptraj_modes(modes_filename, aa3);
neq_pond_1 = readdlm(pond_file)[:, 2];

In [7]:
i = Array{Float64, 2}(aa3, aa3)
i_ref = Ref{Array{Float64, 2}}(i)

j = Array{Float64, 2}(aa3, aa3)
j_ref = Ref{Array{Float64, 2}}(j)
    
ind = Array{Int64}(aa3)
ind_ref = Ref{Array{Int64, 1}}(ind)

aa3_ref = Ref{Int64}(aa3)

ccall( (:apc_wrapper_, "./apc_wrapper.so"), Void,
(Ref{Int64}, Ref{Int64},
Ref{Array{Float64, 2}}, Ref{Array{Float64, 2}}, Ref{Int64}),
aa3_ref, aa3_ref, i_ref, j_ref, ind)

In [ ]:
aa3_ref[]

In [15]:
nuevo = Array{Int64}(aa3)
ccall( (:apc_wrapper_, "./2apc_wrapper.so"), Void,
(Ref{Int64}, Ref{Int64},
Ref{Array{Float64, 2}}, Ref{Array{Float64, 2}}, Ref{Int64}),
aa3_ref, aa3_ref, i_ref, j_ref, nuevo)

In [17]:
nuevo3 = Array{Int64}(aa3)
ccall( (:apc_wrapper_, "./3apc_wrapper.so"), Void,
(Ref{Int64}, Ref{Int64},
Ref{Array{Float64, 2}}, Ref{Array{Float64, 2}}, Ref{Int64}),
aa3_ref, aa3_ref, i_ref, j_ref, nuevo3)

##  Full width version

In [ ]:
# Escribo todo lo correspondiente al 1er frame.
        # Store the current modes, eigenvalues, weights and sorting indices
writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_mode_frame_", 1),
        DataFrame(neq_modes_1), header = false, separator = '\t')
writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_eigen_frame_", 1),
        DataFrame(y=neq_eigen_1), header = false, separator = '\t')
writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_pond_frame_", 1),
    DataFrame(y=neq_pond_1), header = false, separator = '\t')
writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_indices_frame_", 1),
    DataFrame(y=collect(1:aa3_6)), header = false, separator = '\t')

# Ahora lo de no equilibrio
disponible = collect(1:aa3_6)
indices = Array{Int64, 1}(aa3_6)

for i in 2:n_frames
    # Get ready
    println(i)
    pond_file = open(string(main_dir_long, neq_dir, i, neq_pond_template), "r")
    modes_filename = string(main_dir_long, neq_modes_dir, neq_modes_template, i, ".dat")
    disponible = collect(1:aa3_6)
    
    # Get the modes/eigenvalues of the current subspace
    neq_modes, neq_eigen = read_ptraj_modes(modes_filename, aa3);
    # Get the coefficients for the ponderation
    neq_pond = readdlm(pond_file)[:, 2];
    # Get the projections
    pjtions = transpose(neq_modes) * neq_modes_1;
    
    # Get the proper sorting of modes
    for j=1:aa3_6
        top = Int64
        temp_1 = abs(pjtions[:, j])
        index_ord_temp_1 = sortperm(temp_1, rev=true)
        
        for top in index_ord_temp_1
            # Has the maximum overlaping mode already been assigned?
            idx = findin(disponible, top)
            if (length(idx) != 0)
                # No, assign this mode as the corresponding one.
                deleteat!(disponible, idx)
                break;
            end
            # Yes, get the next most overlapping mode and be sure to keep track of 
            # the changes in the indices, due to removal of elements from 'temp_1'.
        end
        
        indices[j] = top
    end

        # Store the current modes, eigenvalues, weights and sorting indices
    writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_mode_frame_", i),
        DataFrame(neq_modes[:, indices]), header = false, separator = '\t')
    writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_eigen_frame_", i),
        DataFrame(y=neq_eigen[indices]), header = false, separator = '\t')
    writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_pond_frame_", i),
    DataFrame(y=neq_pond[indices]), header = false, separator = '\t')
    writetable(string(main_dir_long, min_cost_modes_full_dir, "neq_indices_frame_", i),
        DataFrame(y=indices), header = false, separator = '\t')
end

##  Limited width version

In [ ]:
# Escribo todo lo correspondiente al 1er frame.
        # Store the current modes, eigenvalues, weights and sorting indices
writetable(string(main_dir_long, min_cost_modes_dir, "neq_mode_frame_", 1),
        DataFrame(neq_modes_1), header = false, separator = '\t')
writetable(string(main_dir_long, min_cost_modes_dir, "neq_eigen_frame_", 1),
        DataFrame(y=neq_eigen_1), header = false, separator = '\t')
writetable(string(main_dir_long, min_cost_modes_dir, "neq_pond_frame_", 1),
    DataFrame(y=neq_pond_1), header = false, separator = '\t')
writetable(string(main_dir_long, min_cost_modes_dir, "neq_indices_frame_", 1),
    DataFrame(y=collect(1:aa3_6)), header = false, separator = '\t')



# Ahora lo de no equilibrio
ancho = 10
rango = collect(-ancho:1:ancho)
disponible = collect(1:aa3_6)
indices = Array{Int64, 1}(aa3_6)



for i in 2:n_frames
    # Get ready
    println(i)
    pond_file = open(string(main_dir_long, neq_dir, i, neq_pond_template), "r")
    modes_filename = string(main_dir_long, neq_modes_dir, neq_modes_template, i, ".dat")
    disponible = collect(1:aa3_6)
    
    # Get the modes/eigenvalues of the current subspace
    neq_modes, neq_eigen = read_ptraj_modes(modes_filename, aa3);
    # Get the coefficients for the ponderation
    neq_pond = readdlm(pond_file)[:, 2];
    # Get the projections
    pjtions = transpose(neq_modes) * neq_modes_1;
    
    # Get the proper sorting of modes
    for j=1:aa3_6
        chunk = rango[0 .< (rango.+j) .<= aa3_6 ] .+ j
        n = length(chunk)
        top = Int64
        temp_1 = abs(pjtions[chunk, j])
        index_ord_temp_1 = sortperm(temp_1, rev=true)
    
        if j > (ancho+1)
            # Displace the indices starting @ "ancho+1"
            index_ord_temp_1 = index_ord_temp_1 .+ (j - ancho - 1)
        end
        
        for top in index_ord_temp_1
            # Has the maximum overlaping mode already been assigned?
            idx = findin(disponible, top)
            if (length(idx) != 0)
                # No, assign this mode as the corresponding one.
                deleteat!(disponible, idx)
                break;
            end
            # Yes, get the next most overlapping mode and be sure to keep track of 
            # the changes in the indices, due to removal of elements from 'temp_1'.
        
            if top == index_ord_temp_1[end]
            # These are the last modes, no available replacements
                top = 0
                for ii = 1:length(disponible)
                    tmp = disponible[ii]
                    if abs(pjtions[tmp]) > top
                        top = tmp
                        deleteat!(disponible, ii)
                        break
                    end
                end
            end
        end
        
        indices[j] = top
    end

    # Store the current modes, eigenvalues, weights and sorting indices
    writetable(string(main_dir_long, min_cost_modes_dir, "neq_mode_frame_", i),
        DataFrame(neq_modes[:, indices]), header = false, separator = '\t')
    writetable(string(main_dir_long, min_cost_modes_dir, "neq_eigen_frame_", i),
        DataFrame(y=neq_eigen[indices]), header = false, separator = '\t')
    writetable(string(main_dir_long, min_cost_modes_dir, "neq_pond_frame_", i),
    DataFrame(y=neq_pond[indices]), header = false, separator = '\t')
    writetable(string(main_dir_long, min_cost_modes_dir, "neq_indices_frame_", i),
        DataFrame(y=indices), header = false, separator = '\t')
end